## Spotipy API

Create an Spotify account and follow these steps to register an app: https://developer.spotify.com/documentation/general/guides/app-settings/

After the app is created, you can see it on your dashboard
https://developer.spotify.com/dashboard/applications

Click on it and you'll find the client id and client secret.

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#some extra useful libraries
import pandas as pd
import json
import pprint

In [ ]:
#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id="<introduce your client id>",
    client_secret="<introduce your client secret>"))

In [ ]:
#Quick test - Searching songs with 'queries' with sp.search

results = sp.search(q='Bohemian Rhapsody', limit=50)
results

#(reminder- raw output will be json 
#this feels like a good time to look at the documentation!)

## documentation - and useful links 

developer app 
https://developer.spotify.com/documentation/general/guides/app-settings/

spotipy documentation
https://spotipy.readthedocs.io/en/2.16.1/
    
examples of usage of spotipy including functions 
https://github.com/plamere/spotipy/tree/master/examples 
    
spotify development space with docs 
https://developer.spotify.com/
- the documentation is fully searchable and has neat try it feature (with temporary token), explains the input parameters, clarifies what your query looks like underneath and lists all the key pairs and data in the output with definitions 

AND you can have the spotify web player open https://open.spotify.com/

a) You can start with exploring the results of your query

In [ ]:
#we know that JSON objects are written in key/value pairs.
# so what are the keys of this data set? 
results.keys()

In [ ]:
#what is inside the track key? ie what can I navigate with?
results["tracks"].keys()

In [ ]:
#remember the href ? - thats your query- 
#lets look at what we have searched using the wrapper
results["tracks"]["href"]

In [ ]:
#in tracks - we can get the json on the tracks (its still hard to read)
results["tracks"]["items"] 

In [ ]:
#we did run a limit of 50 in our query - but can easily go to next 50
results["tracks"]["next"]

In [ ]:
#if we want to go back to the prev 50 - shortcut
results["tracks"]["previous"] 

In [ ]:
#total number of matches from the original query string
results["tracks"]["total"]

b) next, we can drill in to one row of your results - ie one track

In [ ]:
#in my example I am using the index value 5, ie the 6th member returned 
results["tracks"]["items"][5].keys() #items (actual tracks)

In [ ]:
#inside the json string we have information about the album for the selected track
results["tracks"]["items"][5]["album"]

In [ ]:
#and the artist
results["tracks"]["items"][5]["artists"]

In [ ]:
#the name of the song 
results["tracks"]["items"][5]["name"]

In [ ]:
# how popular is it on spotify?
results["tracks"]["items"][5]["popularity"]

What is a popularity score? 

The score is received from the Spotify API. The value will be between 0 and 100, with 100 being the most popular.

The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are.

Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time.

In [ ]:
# to get the uri of a song
results["tracks"]["items"][5]["uri"]

Spotify songs are identified by either a "url", a "uri" or an "id". 

- The `id` is an alphanumeric code, and it's the nuclear part of the identifier.

- The `uri` contains "spotify:track" before the id. An uri is useful because it can be searched manually in the Spotify app.

- The `url` is a link to the song on the Spotify web player.

We'll use the `uri` in this code-along, but feel free to use whatever you think fits best your needs.

In [ ]:
#what can we pull out of the api for this track ? 
results["tracks"]["items"][5].keys() 

Artist analysis - this is more just for fun than for the project 

In [ ]:
#get all albums for your favourite artist
#- using spotify web browser to find url of chosen artist 

artist = sp.artist("https://open.spotify.com/artist/1dfeR4HaWDbWqFHLkxsg1d")

albums = []
results = sp.artist_albums(artist['id'], album_type='album')
albums.extend(results['items'])
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])

albums.sort(key=lambda album:album['name'].lower())
for album in albums:
    name = album['name']
    print((' ' + name))

print(len(albums))

In [ ]:
#how to query more than one artist at once
artists = ["Katy Perry", "Duffy", "Adele"]

In [ ]:
my_3_artists = [sp.search(q=artist, limit=50) for artist in artists]

In [ ]:
my_3_artists[0]["tracks"]["items"][7]["uri"]

In [ ]:
#Function to get the artists involved in a song:

def get_artists_from_track(track):
    return [artist["name"] for artist in track["artists"]]

In [ ]:
#here we are returning to using the results set from our earlier query 
#but you could also build upon any other results from the API
my_track = results["tracks"]["items"][5]

In [ ]:
get_artists_from_track(my_track)

In [ ]:
#Function to get the "id's" of the artists from a track:

def get_artists_ids_from_track(track):
    return[artist["id"] for artist in track["artists"]]

In [ ]:
#part 2 of this function 

get_artists_ids_from_track(my_track)

### Playlists

We will need to collect a "database" of songs. Playlists are a good way to access relatively large amounts of songs.

do you already have a playlist of your own you can use ? 

do one of your classmates have a great playlist for this?

or can you find a playlist on spotify which suits your needs ? 

hint: this is a shortcut to doing this part of your MVP

In [ ]:
#I am using an example playlist from spotify - 2020 hits&best music

#read a playlist 

playlist_id = 'spotify:user:spotifycharts:playlist:31ymdYCITDnZRtkKzP3Itp'
results = sp.playlist(playlist_id)
print(json.dumps(results, indent=4))

In [ ]:

#capture features of your own playlists into a df - enter your id and playlist id

playlist = sp.user_playlist("yourid", "playlistid") 
songs = playlist["tracks"]["items"] 
ids = [] 
for i in range(len(songs)): 
    ids.append(songs[i]["track"]["id"]) 
features = sp.audio_features(ids) 
df = pd.DataFrame(features)
df.head()


In [ ]:
#or define from a public playlist

playlist = sp.user_playlist_tracks("spotify", "31ymdYCITDnZRtkKzP3Itp")

In [ ]:
#check its working - first track 
playlist["items"][0]["track"]["uri"]

In [ ]:
for song in playlist["items"]:
    print(song.keys())

In [ ]:
#how many songs on that playlist?
playlist["total"]

In [ ]:
#function to get tracks from playlist 

def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [ ]:
tracks = get_playlist_tracks("31ymdYCITDnZRtkKzP3Itp")

### Audio features

You can check here an explanation of the audio features: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/

In [ ]:
#get the results of a playlist
playlist = sp.user_playlist_tracks("spotify", "36UuuONPIdnKZykWOt2Poz")

In [ ]:
# get the uri of a single song:
song_uri = playlist["items"][0]["track"]["uri"]

In [ ]:
#by the way, what is that song ?
playlist["items"][0]["track"]["name"]

In [ ]:
# get the audio features for that song
sp.audio_features(song_uri)

In [ ]:
#function to extract all Uris from a playlist 
#- we already had the function to take the songs from the playlist 

def get_song_uris(playlist_id):
    tracks = get_playlist_tracks(playlist_id)
    uris = [track["track"]["uri"] for track in tracks]
    return uris

In [ ]:
#part 2 of function

IH_uris = get_song_uris("36UuuONPIdnKZykWOt2Poz")

Above, we stored all the uri's of a playlist into a list called IH_uris. We're going to get all the audio features from that playlist's songs now.

In [ ]:
aud_feat = []

for song in IH_uris:
    aud_feat.append(sp.audio_features(song))

In [ ]:
#take a look at the features of all the songs in the playlist 

aud_feat

### Searching the audio features for a song

When the user inputs a song, you are gonna want to retrieve the audio features of that song. How to do it?

1. Search the user input using the `sp.search()` function. This function works similarly to the "search" bar on the spotify app - using Spotify's intelligent search engine. That means that it can handle names of any songs or artists - even certain typos.

2. Find the uri of the song that the API gives you back.

3. Use `sp.audio_features` to retrieve the audio features of the song.

In [ ]:
#1 
WAP = sp.search(q='WAP', limit=1)

In [ ]:
# 2
usersearch["tracks"]["items"][0]["uri"]

In [ ]:
# 3
sp.audio_features(usersearch["tracks"]["items"][0]["uri"])

### Lab: Create your collection of songs & audio features

To move forward witht the project, you need to create a collection of songs with their audio features - as large as possible! 

These are the songs that we will cluster. And, later, when the user inputs a song, we will find the cluster to which the song belongs and recommend a song from the same cluster.

The more songs you have, the more accurate recommendations you'll be able to give. Although, you might want to make sure the collected songs are "curated" in a certain way. Try to find playlists of songs that are diverse, but also that meet certain standards.

The process of sending hundreds or thousands of requests can take some time - it's normal if you have to wait a few minutes (or, if you're ambitious, even hours) to get all the data you need.

An idea for collecting as many songs as possible is to start with all the songs of a big, diverse playlist and then go to every artist present in the playlist and grab every song of every album of that artist. The amount of songs you'll be collecting per playlist will grow exponentially!